# Hyperparameter optimization `no dia` vs (`pre`, `dia`)

## Init config

In [ ]:
ordinal_orders = {
    "GenHlth": ["excellent", "very good", "good", "fair", "poor"],
    "Age": [
        "18-24",
        "25-29",
        "30-34",
        "35-39",
        "40-44",
        "45-49",
        "50-54",
        "55-59",
        "60-64",
        "65-69",
        "70-74",
        "75-79",
        "80+",
    ],
    "Education": [
        "no school",
        "elementary",
        "some high school",
        "high school graduate",
        "college",
        "college graduate",
    ],
    "Income": ["<$10k", "<$15k", "<$20k", "<$25k", "<$35k", "<$50k", "<$75k", ">$75k"],
}

In [ ]:
import os
from datetime import datetime
import time
import pandas as pd
import numpy as np

import seaborn as sns
from sklearn.metrics import f1_score

from src.config import (
    DATA_SPLIT_DIR,
    TRAIN_RAW_FILENAME,
    VALIDATION_RAW_FILENAME,
    MODELS_DIR,
    STUDY_DIR,
)

os.makedirs(STUDY_DIR, exist_ok=True)

# For Bayesian Optimization
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score, StratifiedKFold

# importing plotly and enable jupyter notebooks for showing optuna visualisations
import plotly.io as pio

pio.renderers.default = "iframe"

# for model comparison
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

from sklearn.exceptions import DataConversionWarning
import warnings

warnings.filterwarnings(action="ignore", category=DataConversionWarning)

## Load train data

In [ ]:
df_train_raw = pd.read_csv(os.path.join(DATA_SPLIT_DIR, TRAIN_RAW_FILENAME))
features_train_raw = df_train_raw.drop("Diabetes_012", axis=1)
target_train_raw = df_train_raw["Diabetes_012"].replace({"pre": "dia"})

## Bayesian Optimization

### Pipeline

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# there are only binary values for the nominal columns
nominal_cols = [
    "HighBP",
    "HighChol",
    "CholCheck",
    "Smoker",
    "Stroke",
    "HeartDiseaseorAttack",
    "PhysActivity",
    "Fruits",
    "Veggies",
    "HvyAlcoholConsump",
    "AnyHealthcare",
    "NoDocbcCost",
    "DiffWalk",
    "Sex",
]
nominal_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first")),
    ]
)

ordinal_cols = ["GenHlth", "Age", "Education", "Income"]

ordinal_categories = [ordinal_orders[col] for col in ordinal_cols]

ordinal_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("encode", OrdinalEncoder(categories=ordinal_categories)),
    ]
)


numeric_cols = ["BMI", "MentHlth", "PhysHlth"]
num_pipe = Pipeline(
    [("impute", SimpleImputer(strategy="median")), ("scale", StandardScaler())]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("ord", ordinal_pipe, ordinal_cols),
        ("nom", nominal_pipe, nominal_cols),
    ],
    remainder="drop",
)

pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("smote", SMOTE(random_state=5)),
    ]
)

In [ ]:
#preprocessed_features_train, sampled_target_train
sampled_features_train, sampled_target_train = pipeline.fit_resample(features_train_raw, target_train_raw)


### Define Study

In [ ]:
model_purpose = "nodia-vs-dia"
study_purpose = f"rf,{model_purpose}"

In [ ]:
%%time
# Bayesian Optimization
def objective(trial):
    """return maximized f1-score"""

    # search space

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 250, step=20),
        "max_features": trial.suggest_categorical(
            "max_features", choices=['sqrt', 'log2']
        ),
        "max_depth": trial.suggest_int("max_depth", 5, 60, step=5),
        "min_samples_split": trial.suggest_int(
            name="min_samples_split", low=2, high=102, step=5
        ),
        "min_samples_leaf": trial.suggest_int(
            name="min_samples_leaf", low=1, high=101, step=5
        ),
    }

    # random forest classifier object
    model_rf = RandomForestClassifier(
        class_weight="balanced",
        random_state=42,
        **params,
    )

    # initiating cv
    scores = cross_val_score(
        estimator=model_rf,
        X=sampled_features_train,
        y=sampled_target_train,
        scoring="f1_macro",
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
    )

    scores = [score for score in scores if not np.isnan(score)]

    return np.mean(scores)

In [ ]:
timestamp = datetime.now()
study_timestamp_str = timestamp.strftime("%Y%m%d%H%M%S")

study_name = f"{study_timestamp_str}_{study_purpose}"



# create a study (aim to maximize score) und setting a seed (random_state) for reproduceability
study = optuna.create_study(
    sampler=TPESampler(seed=42),
    direction="maximize",
    study_name=study_name,
    storage=f"sqlite:///{os.path.join(STUDY_DIR, study_name)}.db",
)

# perform hyperparamter tuning (while timing the process)
time_start = time.time()

### Run

In [ ]:
%%time
study = optuna.load_study(
    study_name=study_name,
    storage=f"sqlite:///{os.path.join(STUDY_DIR, study_name)}.db"
)
study.optimize(objective, n_trials=20)

#### Best parameters

In [ ]:
study.best_params

### Retrain

In [ ]:
%%time
best_params = study.best_params

classifier = RandomForestClassifier(class_weight="balanced", random_state=42, **best_params)

classifier.fit(sampled_features_train, sampled_target_train)

model_timestamp = datetime.now()

In [ ]:
type(datetime.now())

## Prediction

In [ ]:
df_val = pd.read_csv(os.path.join(DATA_SPLIT_DIR, VALIDATION_RAW_FILENAME))
features_val = df_val.drop("Diabetes_012", axis=1)
target_val = df_val["Diabetes_012"].replace({"pre": "dia"})

target_val_pred = classifier.predict(preprocessor.transform(features_val))

## Evaluation

### Metrics

In [ ]:
special_features = "smote"

In [ ]:
from src.model_evaluation import (
    evaluate_classifier,
    get_classification_report_from_results_as_df,
    get_confusion_matrix_from_results_as_df,
)

target_val_pred_proba = None

if hasattr(classifier, "predict_proba"):
    target_val_pred_proba = classifier.predict_proba(preprocessor.transform(features_val))

    if target_train_raw.nunique() <= 2:
        target_val_pred_proba = target_val_pred_proba[:, 1]

# comprehension is necessary to keep the order
labels = [n for n in ["no dia", "pre", "dia"] if n in target_train_raw.unique()]

results = evaluate_classifier(
    classifier=classifier,
    labels=labels,
    target_truth=target_val,
    target_pred=target_val_pred,
    target_pred_proba=target_val_pred_proba,
    timestamp=model_timestamp,
    model_purpose=model_purpose,
    special_features=special_features,
)


print("precision:", results["precision"])
print("F1:", results["f1"])
print("bal acc:", results["bal_accuracy"])
print("roc auc:", results["roc_auc_score"])

print()
print("confusion_matrix")
display(get_confusion_matrix_from_results_as_df(results))

print()
print("classification_report")
display(get_classification_report_from_results_as_df(results))


labels = results["predicts"]
model_name = results["model_name"]

### Save the model and results

In [ ]:
import pickle
import json
import os

folder = os.path.join(MODELS_DIR, model_name)
filename = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

with open(f"{filename}.model.pkl", "wb") as f:
    pickle.dump(classifier, f)

with open(f"{filename}.pipeline.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

with open(f"{filename}.model.txt", "w") as file:
    file.write(str(classifier))

with open(f"{filename}.results.json", "w") as f:
    json.dump(results, f, indent=2)

with open(f"{filename}.pipeline_params.txt", "w") as f:
    f.write(preprocessor.get_params().__str__())

with open(f"{filename}.model_params.json", "w") as f:
    json.dump(classifier.get_params(), f, indent=2)

### Learning curves

In [ ]:
%%time

scoring_mode = "f1_macro"
train_sizes, train_scores, val_scores = learning_curve(
    estimator=classifier,
    X=preprocessor.transform(features_train_raw),
    y=target_train_raw,
    cv=5,
    scoring=scoring_mode,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
)

train_mean = train_scores.mean(axis=1)
val_mean = val_scores.mean(axis=1)

df_lc = pd.DataFrame({"train_sizes": train_sizes, "train_scores": train_mean, "val_scores": val_mean})
df_lc.to_csv(f"{filename}.learning_curves.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import io
from scipy.ndimage import uniform_filter1d


fig, ax = plt.subplots(figsize=(16, 5))

# Hauptlinien
train_line, = ax.plot(train_sizes, train_mean, label="Training score")
val_line, = ax.plot(train_sizes, val_mean, label="Validation score")

# Farben extrahieren
train_color = train_line.get_color()
val_color = val_line.get_color()

# Min-/Max-Werte
y_lines_min = [train_mean.min(), val_mean.min()]
y_lines_max = [train_mean.max(), val_mean.max()]

# Hilfslinien
for y in y_lines_min:
    ax.axhline(y, color="red", linestyle="--", linewidth=0.8, alpha=0.2)
for y in y_lines_max:
    ax.axhline(y, color="green", linestyle="--", linewidth=0.8, alpha=0.2)

# Rollender Durchschnitt (Fenstergröße = 3)
train_rolling = uniform_filter1d(train_mean, size=3, mode="nearest")
val_rolling = uniform_filter1d(val_mean, size=3, mode="nearest")

ax.plot(train_sizes, train_rolling, label="Train rolling avg", linestyle=":", linewidth=1.5, color=train_color, alpha=0.5)
ax.plot(train_sizes, val_rolling, label="Val rolling avg", linestyle=":", linewidth=1.5, color=val_color, alpha=0.5)

# y-Ticks
yticks = sorted(set(ax.get_yticks().tolist() + y_lines_min + y_lines_max))
ax.set_yticks(yticks)

ax.set_xlabel("Training set size")
ax.set_ylabel("F1 Macro Score")

ax.set_title(f"Learning curve for model '{model_name}'")

ax.legend()
fig.tight_layout()
fig.savefig(f"{filename}.learning_curve_img.png", format="png")
plt.show()

### Feature importances

In [ ]:
preprocessed_features_train = preprocessor.transform(features_train_raw)
tgt_train_pred = classifier.predict(preprocessed_features_train)

f1_base = f1_score(target_train_raw, tgt_train_pred, average='weighted')

importances_of_permutations = []
for feat in features_train_raw.columns:
    features_permutated = features_train_raw.copy()

    permutation_run_scores = []
    for i in range(5):
        series_perm = features_permutated[feat].sample(frac=1, replace=False, random_state=0)
        series_perm.reset_index(drop=True, inplace=True)
        features_permutated[feat] = series_perm
        
        preprocessed_features_permutated = preprocessor.transform(features_permutated)
        f1_permutated = f1_score(target_train_raw, classifier.predict(preprocessed_features_permutated), average='weighted')
        permutation_run_scores.append(f1_base - f1_permutated)

    importances_of_permutations.append(np.mean(permutation_run_scores))

feature_importances = pd.Series(data=importances_of_permutations, index=features_train_raw.columns)
feature_importances.sort_values(ascending=False, inplace=True)

with open(f"{filename}.feature_importances.json", "w") as f:
    json.dump(feature_importances.to_dict(), f, indent=2)


In [ ]:
fig, ax = plt.subplots(figsize=(16, 0.3*len(features_val.columns)))
sns.barplot(x=feature_importances.values, y=feature_importances.index,ax=ax)
ax.set_title(f"feature importances for model '{model_name}'")
for i, (val, label) in enumerate(zip(feature_importances.values, feature_importances.index)):
    ax.text(val, i, f" {val:.4f}", va="center", ha="left", fontsize=8)
    
fig.tight_layout()
fig.savefig(f"{filename}.feature_importances.png", format="png")
plt.show()

## Optuna Vizualizations

### History

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

### Parallel coordinates

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

### Parameter importances

In [ ]:
optuna.visualization.plot_param_importances(study)